In [3]:
import requests
r = requests.get('http://api.census.gov/data/2011/acs5?get=B08126_033E,B08126_018E,B08126_048E&for=tract:060800&in=state:08+county:013&key=2bf6c18a5dd3a9af0a17bd4f8b5528f192a34174')
r.json()

[['B08126_033E', 'B08126_018E', 'B08126_048E', 'state', 'county', 'tract'],
 ['57', '188', '0', '08', '013', '060800']]

In [116]:
import pandas as pd
import numpy as np
import pydot
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz

In [78]:
def get_census_data():
    return pd.read_csv('data.csv', index_col=0)
df = get_census_data()
df = df.apply(lambda x: x.replace('.',''))
df = df.drop(' Government', 1)
df = df.drop(' x', 1)
df = df.drop(' y', 1)
df = df.drop(' z', 1)
df = df.drop(' Number', 1)
df.head(10)
df.columns

Index([' Education', ' Years in workforce', ' Marital Status', ' Occupation',
       ' Familial Role', ' Race', ' Gender', ' country', ' Generic income'],
      dtype='object')

In [40]:
df.groupby([" Occupation"]).groups.keys()

dict_keys([' Exec-managerial', ' Farming-fishing', ' Transport-moving', ' Sales', ' Craft-repair', ' Prof-specialty', ' Adm-clerical', ' Priv-house-serv', ' Handlers-cleaners', ' Machine-op-inspct', ' Armed-Forces', ' Protective-serv', ' Tech-support', ' Other-service', ' ?'])

In [98]:
mapper_list = []
def encode(df):
    df_mod = df.copy()
    for column in df:
        df_mod = df_mod.copy()
        targets = df_mod[column].unique()
        map_to_int = {name: n for n, name in enumerate(targets)}
        mapper_list.append(map_to_int)
        df_mod[column] = df_mod[column].replace(map_to_int)
    return df_mod
df_mod = encode(df)
df_mod.head(10)

,Education,Years in workforce,Marital Status,Occupation,Familial Role,Race,Gender,country,Generic income
ID,,,,,,,,,
39,0,0,0,0,0,0,0,0,0
50,0,0,1,1,1,0,0,0,0
38,1,1,2,2,0,0,0,0,0
53,2,2,1,2,1,1,0,0,0
28,0,0,1,3,2,1,1,1,0
37,3,3,1,1,2,0,1,0,0
49,4,4,3,4,0,1,1,2,0
52,1,1,1,1,1,0,0,0,1
31,3,3,0,3,0,0,1,0,1


In [86]:
features = list(df_mod.columns[0:8])
y = df_mod[' Generic income']
x = df_mod[features]
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best')

In [138]:
predictions = dt.predict(df_mod.as_matrix(df_mod.columns[0:8]))
results = ['<=50k' if not i else '>50k' for i in predictions]
results

['<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '>50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '<=50k',
 '>50k',
 '<=50k',
 '<=50k',
 '<=50k',
 '>50k',
 '>50k',
 '<=50k

In [111]:
def visualize_tree(tree, feature_names):
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)
    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    subprocess.check_call(command)

In [121]:
with open('tree.dot', 'w') as dotfile:
    export_graphviz(
        dt,
        dotfile,
        feature_names=[' Education', ' Years in workforce', ' Marital Status', ' Occupation', ' Familial Role', ' Race', ' Gender', ' country'])
    

In [131]:
mapper_list


[{' 10th': 12,
  ' 11th': 2,
  ' 12th': 15,
  ' 1st-4th': 13,
  ' 5th-6th': 11,
  ' 7th-8th': 8,
  ' 9th': 4,
  ' Assoc-acdm': 6,
  ' Assoc-voc': 7,
  ' Bachelors': 0,
  ' Doctorate': 9,
  ' HS-grad': 1,
  ' Masters': 3,
  ' Preschool': 14,
  ' Prof-school': 10,
  ' Some-college': 5},
 {1: 14,
  2: 13,
  3: 11,
  4: 8,
  5: 4,
  6: 12,
  7: 2,
  8: 15,
  9: 1,
  10: 5,
  11: 7,
  12: 6,
  13: 0,
  14: 3,
  15: 10,
  16: 9},
 {' Divorced': 2,
  ' Married-AF-spouse': 5,
  ' Married-civ-spouse': 1,
  ' Married-spouse-absent': 3,
  ' Never-married': 0,
  ' Separated': 4,
  ' Widowed': 6},
 {' ?': 11,
  ' Adm-clerical': 0,
  ' Armed-Forces': 13,
  ' Craft-repair': 6,
  ' Exec-managerial': 1,
  ' Farming-fishing': 8,
  ' Handlers-cleaners': 2,
  ' Machine-op-inspct': 9,
  ' Other-service': 4,
  ' Priv-house-serv': 14,
  ' Prof-specialty': 3,
  ' Protective-serv': 12,
  ' Sales': 5,
  ' Tech-support': 10,
  ' Transport-moving': 7},
 {' Husband': 1,
  ' Not-in-family': 0,
  ' Other-relative': 